<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-soyoungcareer/blob/main/week3/Chapter2_1_%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 과제 수행 순서
    1. Tokenizer 및 `TextClassifier`의 encoder를 GPT로 변경
        1. GPT 로 변경
        2. UNK token 추가
    2. attention_mask 제거
    3. representation token 변경


- 질문 사항
    - 기본과제와 마찬가지로 Train Loss가 줄어드는 듯 하다가 줄지 않는 이유를 모르겠음.

In [2]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00


In [3]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

# 1-a. GPT 로 변경
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'openai-gpt')
# 1-b. UNK token 추가 -> GPT에는 padding token 이 없음.
tokenizer.pad_token = tokenizer.unk_token

/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://h

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
ds = load_dataset("fancyzhx/ag_news")

# 라벨의 고유 값 확인
train_labels = ds['train']['label']
unique_labels = set(train_labels)

print(f"Number of unique labels: {len(unique_labels)}")
print(f"Unique labels: {unique_labels}")

for example in list(ds['train'])[:5]:
    print(f"Text: {example['text']}\nLabel: {example['label']}")


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  encodings = tokenizer(texts, padding=True, return_tensors="pt")
  texts = encodings.input_ids

  # 2. GPT 모델에서는 attention_mask 를 사용하지 않음.
  labels = torch.LongTensor(labels)

  return texts, labels

batch_size = 64

train_loader = DataLoader(
    ds['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=batch_size, shuffle=False, collate_fn=collate_fn
)

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Number of unique labels: 4
Unique labels: {0, 1, 2, 3}
Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Label: 2
Text: Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
Label: 2
Text: Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
Label: 2
Text: Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Sa

In [5]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'openai-gpt')    # 1-a. GPT 로 변경
model

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

OpenAIGPTModel(
  (tokens_embed): Embedding(40478, 768)
  (positions_embed): Embedding(512, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x Block(
      (attn): Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
  )
)

In [6]:
from torch import nn


class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'openai-gpt')    # 1-a. GPT 로 변경
    self.dropout = nn.Dropout(0.1)    # 과적합 방지
    self.classifier = nn.Linear(768, 4)

  def forward(self, input_ids):
    # 2. GPT 모델은 attention_mask를 사용하지 않음.
    x = self.encoder(input_ids)['last_hidden_state']

    # 3. 마지막 토큰의 representation 사용 -> GPT 모델은 마지막 토큰에 가장 많은 정보가 담겨 있음.
    x = self.dropout(x[:, -1])
    x = self.classifier(x)

    return x


model = TextClassifier()

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [7]:
for param in model.encoder.parameters():
  param.requires_grad = False

In [8]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)
# scheduler = StepLR(optimizer, step_size=3, gamma=0.1)  # 학습률 스케줄러: 3에포크마다 학습률을 0.1배로 줄임
n_epochs = 10

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = (
        inputs.to('cuda'),
        labels.to('cuda')
    )

    # preds = model(inputs)[..., 0]
    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    # print("preds.shape : ", preds.shape)
    # print("labels.shape : ", labels.shape)

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 1386.828848272562
Epoch   1 | Train Loss: 1211.0923843681812
Epoch   2 | Train Loss: 1201.0801371335983
Epoch   3 | Train Loss: 1204.4919863194227
Epoch   4 | Train Loss: 1199.7590579390526
Epoch   5 | Train Loss: 1193.2770167291164
Epoch   6 | Train Loss: 1195.8141517937183
Epoch   7 | Train Loss: 1190.2569801807404
Epoch   8 | Train Loss: 1194.059815198183
Epoch   9 | Train Loss: 1201.5277989208698


=> 학습 시 Train Loss 가 제대로 줄어들지 않음.

In [9]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = (
        inputs.to('cuda'),
        labels.to('cuda')
    )

    preds = model(inputs)
    # 2-c. 다중분류 차원 맞춰줌.
    preds = torch.argmax(preds, dim=-1)
    # preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")


=========> Train acc: 0.834 | Test acc: 0.826


=> 이번 과제를 수행하며 BERT 모델과 GPT 모델의 차이점에 대해 알게 되었음.



#BERT 모델과 GPT 모델의 차이점

###BERT 모델
- 양방향(Bidirectional) 모델로 각 토큰이 양쪽 방향을 모두 참고하여 의미를 학습함.
- CLS 토큰의 hidden state 를 사용하여 예측함. (CLS 토큰이 문장의 정보를 요약하여 가지고 있음)
- Attention Mask 사용. (짧은 문장에 padding token 을 붙여 동일한 길이로 맞춤.)

###GPT 모델
- 단방향(Autoregressive) 모델로 문장을 순차적으로 학습함. 각 토큰이 이전의 토큰들만 참고해서 다음 토큰을 예측함.
- CLS 토큰 없음. 마지막 토큰의 hidden state 를 사용하여 예측함. (마지막 토큰이 중요한 정보를 가지고 있음)
- Attention Mask 사용 안 함. (순차적으로 학습하기 때문에 패딩된 부분을 무시할 필요가 없음)
